This tutorial introduces algorithms for federated learning in FedJAX. By completing this tutorial, you'll learn how to write clear and efficient algorithms that follow best practices. It assumes that the readers have finished the tutorial on datasets and models. 

In order to keep the code pseudo-code-like, we avoid using jax primitives directly while writing algorithms, with the notable exceptions of `jax.random`and `jax.tree_util` libraries. Since lower-level functions that are described in the model tutorial such as `fedjax.optimizers`, `model.grad` are all JIT compiled, the algorithms will still be efficient.

In [ ]:
import jax.numpy as jnp
import jax.random as random
import jax.tree_util as tree_util
import numpy as np

import fedjax
import fedjax.google.datasets as fedjax_data
# We only use TensorFlow for datasets, so we restrict it to CPU only to avoid
# issues with certain ops not being available on GPU/TPU.
fedjax.training.set_tf_cpu_only()

# Introduction

A federated algorithm trains a machine learning model over data distributed over several clients. At a high level, the server server randomly initializes the model parameters and other learning components. Then at each round it does the following:
1. **Client selection**: Select a few clients at each round, typically at random.
2. The server transmits the model parameters and other necessary components to the selected clients.
3. **Client update**: The clients update the model parameter using a subroutine, which typically involves a few epochs of SGD.
4. The clients transmit the updates to the server. 
5. **Server aggregation**: The server combines the updated model parameters and produces new model parameters. 

A pseudo code for a common federated learning algorithm can be found in Algorithm 1 in [Kairouz et al. (2020)](https://arxiv.org/pdf/1912.04977.pdf).
Since FedJAX focuses on federated simulation and there is no actual transmission between clients and the server, we only focus on steps 1, 3, and 5, and ignore steps 2 and 4.
Before we describe each of the modules, we will first describe how to use algorithms that are implemented in FedJAX. 

# Algorithm overview

We implement federated learning algorithms using the `FederatedAlgorithm` interface. The `FederatedAlgorithm` interface has two functions `init` and `apply`. Broadly our implementation has three parts.
1. `ServerState`: This contains all the information available at the server at any given round. It includes model parameters and can also include other parameters that are used during optimization. At every round, a subset of `ServerState` is passed to the clients for federated learning. `ServerState` is also used in checkpointing and evaluation. Hence it is crucial that all the parameters that are modified during the course of the federated learning is stored as part of the `ServerState`. We recommend that `FederateAlgorithm` does not have any mutable parameters to ensure that it performs well with jax.

2. `init`: Initializes the server state.

3. `apply`: Takes a set of client_ids, corresponding datasets, and random keys together with `ServerState` and returns a new `ServerState` along with any information we need from the clients in the form of `client_diagnostics`.

We demonstrate `FederatedAlgorithm` using the federated averraging implementation and emnist dataset. We first initialize the model, datasets and the federated algorithm. 

In [ ]:
train, test = fedjax_data.emnist.load_data(only_digits=False,
                                  mode='sqlite')
model = fedjax.models.emnist.create_conv_model(only_digits=False)

rng = random.PRNGKey(0)
init_model_params = model.init(rng)
# Federated algorithm requires a gradient function, a client optimizer, 
# a server optimizers, and parametes for batching at the client level.
grad_fn = fedjax.model_grad(model)
client_optimizer = fedjax.optimizers.sgd(0.1)
server_optimizer = fedjax.optimizers.sgd(1.0)
batch_hparams = fedjax.ShuffleRepeatBatchHParams(batch_size=10)
fed_alg = fedjax.algorithms.fed_avg.federated_averaging(grad_fn,
                                                        client_optimizer,
                                                        server_optimizer,
                                                        batch_hparams)

We also note that similar to the rest of the library, we only pass on the necessary functions and parameters to the federated algorithm. Hence, to initialize the federated algorithm we only passed the `grad_fn` and did not pass the entire model. With this, we now initialize the server state.

In [ ]:
init_server_state = fed_alg.init(init_model_params)

To run the federated algorithm, we pass the client data and server state to the `apply` function. For this end, we pass client data as a tuple of client id, client data, and the random keys. Adding client ids and random keys has multiple advantages. Firstly client client ids allows to track client diagnostics and would be helpful in debugging. Passing random keys would ensure deterministic execution and allow repeatability. Furthermore, as we discuss later it would help us with fast implementations. We first format the data in this necessary format and then run one round of federated learning.

In [ ]:
# Select 10 client_ids and their data
client_ids = list(train.client_ids())[0:5]
clients_ids_and_data = list(train.get_clients(client_ids))

client_inputs = []
for i in range(5):
  rng, use_rng = random.split(rng)
  client_id, client_data = clients_ids_and_data[i]
  client_inputs.append((client_id, client_data, use_rng))

updated_server_state, client_diagnostics = fed_alg.apply(init_server_state,
                                                         client_inputs)
# Prints the l2 norm of gradients as part of client_diagnostics. 
print(client_diagnostics)

As we see above, the client statistics provide the `delta_l2_norm` of gradient for each client, which can be potentially used for debugging purposes. 

With this background on how to use existing implementations, we are now going to describe how to write your own federated algorithms in FedJAX. This involves three steps: client selection, client update, and server aggregation.

# Client selection

At each round of federated learning, typically clients are sampled uniformly at random. This can be done using numpy as follows. 

In [ ]:
all_client_ids = list(train.client_ids())
print("Total number of client ids: ", len(all_client_ids))

sampled_client_ids = np.random.choice(all_client_ids, size=2, replace=False)
print("Sampled client ids: ", sampled_client_ids)

However, the above code is not desirable due to the following reasons:
1. For reproducibility, it is desirable to have a fixed seed just for sampling clients.
2. Across rounds, different clients need to be sampled.
3. For efficiency reasons, it might be better to do an approximately uniform sampling, where clients whose data is stored together are sampled together.
4. Federated algorithms typically require additional randomness for batching, or dropout that needs to be sent to clients. 

To incorporate these features, we provide a few client samplers.
1. UniformShuffledClientSampler
2. UniformGetClientSampler

UniformShuffledClientSampler is preferred for efficiency reasons, but if we need to sample clients truly randomly, UniformGetClientSampler can be used.
Both of them return a list of client_ids, client_data, and client_rng.

In [ ]:
efficient_sampler = fedjax.client_samplers.UniformShuffledClientSampler(
    train.shuffled_clients(buffer_size=100), num_clients=2)
print("Sampling from the efficient sampler.")
for round in range(0, 3):
  sampled_clients_with_data = efficient_sampler.sample()
  for client_id, client_data, client_rng in sampled_clients_with_data:
    print(round, client_id)

perfect_uniform_sampler = fedjax.client_samplers.UniformGetClientSampler(
    train, num_clients=2, seed=1)
print("Sampling from the perfect uniform sampler.")
for round in range(0, 3):
  sampled_clients_with_data = perfect_uniform_sampler.sample()
  for client_id, client_data, client_rng in sampled_clients_with_data:
    print(round, client_id)

 # Client update

After selecting the clients, the next step would be running a model update step in the clients. Typically this is done running a few epochs of SGD. We only pass parts of the algorithm that are necessary for client update.

The client update typically requires a set of parameters from the server (`init_params` in this example), the client dataset and a source of randomness (`rng`). The randomness can be used for dropout or other model update steps. Finally, instead of passing the entire model to the client update, since our code only depends on the gradient function, we pass `grad_fn` to client_update.

In [ ]:
def client_update(init_params, client_dataset, client_rng, grad_fn):
  opt_state = client_optimizer.init(init_params)
  params = init_params
  for batch in client_dataset.shuffle_repeat_batch(batch_size=10):
    client_rng, use_rng = random.split(client_rng)
    grads = grad_fn(params, batch, use_rng)
    opt_state, params = client_optimizer.apply(grads, opt_state, params)
  delta_params = tree_util.tree_multimap(lambda a, b: a - b,
                                              init_params, params)
  return delta_params, len(client_dataset)

client_sampler = fedjax.client_samplers.UniformGetClientSampler(
    train, num_clients=2, seed=1)
sampled_clients_with_data = perfect_uniform_sampler.sample()
for client_id, client_data, client_rng in sampled_clients_with_data:
  delta_params, num_samples = client_update(init_params,client_data, 
                                            client_rng, grad_fn)
  print(client_id, num_samples, delta_params.keys())

# Server aggregation 

The outputs of the clients are typically aggregated by computing the mean. This can be easily done by using the `tree_mean` function.



In [ ]:
sampled_clients_with_data = perfect_uniform_sampler.sample()
client_updates = []
for client_id, client_data, client_rng in sampled_clients_with_data:
  delta_params, num_samples = client_update(init_params, client_data,
                                            client_rng, grad_fn)
  client_updates.append((delta_params, num_samples))
updated_output = fedjax.tree_util.tree_mean(client_updates)
print(updated_output.keys())

Combing the above steps gives federated algorithm, which can be found in the [example implementation of federated averaging.](https://github.com/google/fedjax/blob/main/examples/fed_avg.py).


# Efficient implementation

The above implementation would be efficient for running on single machines. JAX provides primitives such as `pmap` and `vmap` for efficient parallelization across many accelerators. We provide support for them in federated learning by distributing client computation across several accelerators.

To take advantage of the faster implementation, one needs to implement `client_update` in a specific format. It has three functions `client_init`, `client_step`, and `client_final`, which we describe below.





 ## `client_init`

This function takes the inputs from the server and outputs a `client_step_state` which will be passed in between rounds. It is desirable for the `client_ste_state` to be a dictionary. In this example, it just copies the parameters, optimizer_state and the current state of client randomness. 

In [ ]:
def client_init(server_params, client_rng):
  opt_state = client_optimizer.init(server_params)
  client_step_state = {
      'params': server_params,
      'opt_state': opt_state,
      'rng': client_rng,
  }
  return client_step_state

## `client_step`

`client_step` takes the current step state and a batch of examples and updates the client_step_state. In this example, we run one step of SGD using the batch of examples and update client_step_state to reflect the new parameters, optimization state and randomness.

In [ ]:
def client_step(client_step_state, batch):
  rng, use_rng = random.split(client_step_state['rng'])
  grads = grad_fn(client_step_state['params'], batch, use_rng)
  opt_state, params = client_optimizer.apply(grads,
                                              client_step_state['opt_state'],
                                              client_step_state['params'])
  next_client_step_state = {
      'params': params,
      'opt_state': opt_state,
      'rng': rng,
  }
  return next_client_step_state

## `client_final`

`client_final` modifies the final_client_step_state and returns the desired parameter. In this example, we compute the difference between initial parameters and the final updated parameters in the `client_final` function.

In [ ]:
def client_final(server_params, client_step_state):
  delta_params = tree_util.tree_multimap(lambda a, b: a - b,
                                         server_params,
                                         client_step_state['params'])
  return delta_params

## `for_each_client`

Once we have the three components, we can combine them to create a client_update function using the `for_each_client` function. `for_each_client` returns a function that can be used to run client updates. The sample usage is below.

In [ ]:
for_each_client_update = fedjax.for_each_client(client_init,
                                                client_step,
                                                client_final)

client_sampler = fedjax.client_samplers.UniformGetClientSampler(
    train, num_clients=2, seed=1)
sampled_clients_with_data = perfect_uniform_sampler.sample()
batched_clients_data = [
      (cid, cds.shuffle_repeat_batch(batch_size=10), crng)
      for cid, cds, crng in sampled_clients_with_data
  ]
for client_id, delta_params in for_each_client_update(init_params,
                                                      batched_clients_data):
  print(client_id, delta_params.keys())


We note that`for_each_client_update` requires the client data to be already batched. This is necessary for performance gains while using multiple accelerators. Furthermore, the batch size needs to be the same across all clients.

By default `for_each_client` selects the standard JIT backend. To enable parallelism with TPUs or for debugging one can set it using `fedjax.set_for_each_client_backend(backend)`, where `backend` is either of 'pmap' or 'debug'.

For each client function can also be used to add some additional step wise results, which can be used for debugging. This requires changing `client_step` function.



In [ ]:
def client_step_with_log(client_step_state, batch):
  rng, use_rng = random.split(client_step_state['rng'])
  grads = grad_fn(client_step_state['params'], batch, use_rng)
  opt_state, params = client_optimizer.apply(grads,
                                              client_step_state['opt_state'],
                                              client_step_state['params'])
  next_client_step_state = {
      'params': params,
      'opt_state': opt_state,
      'rng': rng,
  }
  grad_norm = fedjax.tree_util.tree_l2_norm(grads)
  return next_client_step_state, grad_norm


for_each_client_update = fedjax.for_each_client(client_init,
                                                      client_step_with_log,
                                                      client_final,
                                                      with_step_result=True)


for client_id, delta_params, grad_norms in for_each_client_update(init_params,
                                                                  batched_clients_data):
  
  print(client_id, list(delta_params.keys()))
  print(client_id, np.array(grad_norms))

# Recap

In this tutorial, we have covered the following:

-   Using exisiting algorithms.
-   Adding new algorithms.
-   Efficient implementation in the presence of accelerators.